## Notebook analysing data printed by the tested android device at TÜV-Nord
The .txt files in the data directory contain printed information of an android device. This needs to be categorized, clustered and finally analyzed and compared to the _VBOX_ data in the **CrashAnalytics** Notebook.